# Dask and kugupu

The generation of the coupling matrix is the most time consuming calculation done in kugupu.
Luckily as there are no dependencies in calculating different frames, the problem is embarrassingly parallel.
Kugupu is able to use the `dask.distributed` package to calculate many frames in parallel,
as show in this notebook

In [ ]:
import kugupu as kgp
import MDAnalysis as mda
from dask import distributed

First we load our simulation data as normal:

In [ ]:
u = mda.Universe('./datafiles/C6.data', './datafiles/C6.dcd')

u.add_TopologyAttr('names')
namedict = {
    1.008: 'H',
    12.011: 'C',
    14.007: 'N',
    15.999: 'O',
    32.06: 'S',
}
for m, n in namedict.items():
    u.atoms[u.atoms.masses == m].names = n

Now now create a `distributed.Client` to assign the work to.
Here we create a Client running on our local machine, however it is also possible to use a much more powerful Client.

In [ ]:
c = distributed.Client()
c

The generation of results uses the same function,
but we include the `client=` keyword to make the calculation happen in parallel.

In [ ]:
res = kgp.coupling_matrix(u, 5.0, 'lumo', degeneracy=1, client=c)

The results generated this way are identical to as before, and can be saved and used is subsequent analysis as normal.